In [ ]:
import os, yaml, sys
import numpy as np
from sklearn.decomposition import IncrementalPCA
from torchvision.models.feature_extraction import (
    create_feature_extractor,
    get_graph_node_names,
)
import torch 
import cv2
import joblib

ENV = os.getenv("MY_ENV", "dev")
with open("../../config.yaml", "r") as f:
    config = yaml.safe_load(f)
paths = config[ENV]["paths"]
sys.path.append(paths["src_path"])
from general_utils.utils import print_wise, get_layer_output_shape, get_device
from image_processing.utils import concatenate_frames_batch, shuffle_frames, list_videos, get_frames_number, split_in_batches


## Steps
(parallel over the layers of the video)
1. Load videos until you get approx 2 or 3 times the batch-size (or more?) (for very long vids (some of the ones with arcaro for instance - but check), just take the first 20 seconds or so...) (can we estimate the optimal order based on get_video_dimensions?) -> then shuffle, split evenly the frames and pass it to iPCA 
2. Normalize, format and shuffle them
    - If gaze dependent, load gaze, upsample (in time) video and extract gaze-dep spatial window
3. Compute iPCA
4. Save eigenvectors and eigenvalues


In [11]:
from torchvision import models
n_components = 100
device = get_device()
model_name = "alexnet"
layer_name = "features.7"
model_cls = getattr(models, model_name)
model = model_cls(weights=True).to(device).eval()



/Users/tizianocausin/Desktop/virtual_envs/temporal_context/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:

def ipca_videos(paths, rank, layer_name, model_name, model, n_components, video_type, batches_to_proc, batch_sizes, fn_list, long_vids,  device, vid_duration_lim, new_h=224, new_w=224):
    save_name = (f"{video_type}_{model_name}_{layer_name}_ipca_{n_components}_PCs.pkl")
    path = os.path.join(f"{paths["livingstone_lab"]}/tiziano/models", save_name)
    if os.path.exists(path):
        print_wise(f"{path} already exists")
    else:
        print_wise(f"Fitting PCA for layer: {layer_name}", rank=rank)
        frames_batch = []
        feature_extractor = create_feature_extractor(
            model, return_nodes=[layer_name]
        ).to(device)
        tmp_shape = get_layer_output_shape(feature_extractor, layer_name)
        n_features = np.prod(tmp_shape)  # [C, H, W] -> C*H*W
        n_components_layer = min(n_features, n_components)  # Limit to number of features
        ipca = IncrementalPCA(n_components=n_components_layer, batch_size=batch_sizes[0])
        curr_video_idx = 0
        for idx, curr_batch_size in enumerate(batch_sizes):

            print_wise(f"starting batch {idx}", rank=rank)
            frames_batch, curr_video_idx = concatenate_frames_batch(paths, rank, fn_list, frames_batch, curr_video_idx, idx, batches_to_proc, batch_sizes, new_h, new_w, long_vids, vid_duration_lim)
            frames_batch = shuffle_frames(frames_batch)
            inputs = frames_batch[:curr_batch_size, :, :, :]
            inputs = torch.from_numpy(inputs).float().to(device)
            inputs = inputs.permute(0, 3, 1, 2)
            frames_batch = frames_batch[curr_batch_size:, :, :, :]
            with torch.no_grad():
                feats = feature_extractor(inputs)[layer_name]
                print("feats", feats.shape)
                feats = feats.view(feats.size(0), -1).cpu().numpy()
            ipca.partial_fit(feats)
            # end with torch.no_grad():
        
        #ADD FIX UNCOMMENT THIS LATER !!!!!!!
        #joblib.dump(ipca, path) # better this or pkl?
        print_wise(f"Saved PCA for {layer_name} at {path}", rank=rank)



In [16]:
video_type = "YDX"

max_duration = 20

batch_size = 1100
fn_list = list_videos(paths, video_type)
frames_per_vid, long_vids = get_frames_number(paths, fn_list, max_duration)
batch_sizes = split_in_batches(frames_per_vid, batch_size)
batch_sizes = batch_sizes[:20] #ADD TAKE OFF later
rank = 0
progression = 0 
vid_duration_lim = 20 # sec
frames_passed = 0
batches_to_proc = 4

ipca_videos(paths, rank, layer_name, model_name, model, n_components, video_type, batches_to_proc, batch_sizes, fn_list, long_vids, device, vid_duration_lim, new_h=224, new_w=224)

12:16:02 - rank 0 Fitting PCA for layer: features.7
12:16:02 - rank 0 starting batch 0
12:16:03 - rank 0 YDXJ0100A.MP4 read successfully
12:16:06 - rank 0 YDXJ0100.MP4 read successfully
12:16:08 - rank 0 YDXJ0090A.MP4 read successfully
12:16:09 - rank 0 YDXJ0086A.MP4 read successfully
12:16:09 - rank 0 YDXJ0100B.MP4 read successfully
12:16:12 - rank 0 YDXJ0088.MP4 read successfully
12:16:13 - rank 0 YDXJ0090B.MP4 read successfully
12:16:14 - rank 0 YDXJ0086B.MP4 read successfully
12:16:15 - rank 0 YDXJ0087B.MP4 read successfully
feats torch.Size([1111, 384, 13, 13])
12:16:21 - rank 0 starting batch 1
12:16:25 - rank 0 YDXJ0099.MP4 read successfully
feats torch.Size([1111, 384, 13, 13])
12:16:35 - rank 0 starting batch 2
12:16:36 - rank 0 YDXJ0091B.MP4 read successfully
12:16:36 - rank 0 YDXJ0079_80.MP4 read successfully
12:16:37 - rank 0 YDXJ0096A.MP4 read successfully
feats torch.Size([1111, 384, 13, 13])


KeyboardInterrupt: 

In [7]:
import matplotlib.pyplot as plt
#plt.plot(np.cumsum(ipca.explained_variance_ratio_))
print(ipca.components_.shape)

NameError: name 'ipca' is not defined

In [ ]:
def ipca_core(paths, rank, layer_name, model_name, n_components, model, loader, device):
    save_name = (f"imagenet_val_{model_name}_{layer_name}_pca_model_{n_components}_PCs.pkl")
    path = os.path.join(paths["results_path"], save_name)
    if os.path.exists(path):
        print_wise(f"{path} already exists")
    else:
        print_wise(f"Fitting PCA for layer: {layer_name}", rank=rank)
        feature_extractor = create_feature_extractor(
            model, return_nodes=[layer_name]
        ).to(device)
        tmp_shape = get_layer_output_shape(feature_extractor, layer_name)
        n_features = np.prod(tmp_shape)  # [C, H, W] -> C*H*W
        n_components_layer = min(n_features, n_components)  # Limit to number of features
        pca = IncrementalPCA(n_components=n_components_layer)
        counter = 0
        for inputs, _ in loader:
            counter += 1
            print_wise(f"starting batch {counter}", rank=rank)
            with torch.no_grad():
                inputs = inputs.to(device)
                feats = feature_extractor(inputs)[layer_name]
                feats = feats.view(feats.size(0), -1).cpu().numpy()
                pca.partial_fit(feats)

        joblib.dump(pca, path) # better this or pkl?
        print_wise(f"Saved PCA for {layer_name} at {path}", rank=rank)

